# Building LLM applications: Notebook 03

# Tools

## Initialize

In [1]:
import os
import dotenv
import json

from langchain_core.messages import HumanMessage
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import Runnable, RunnablePassthrough
from langchain_core.tools import tool

from langchain.agents.output_parsers.react_single_input import ReActSingleInputOutputParser

from langchain_ollama import ChatOllama

In [2]:
MODEL = 'llama3.2'

In [ ]:
# Read fro `.env` file
dotenv.load_dotenv()

OLLAMA_URL = os.getenv('OLLAMA_URL')
print(f"Using Ollama server: {OLLAMA_URL if OLLAMA_URL else 'local'}")

## Calling tools manually

### Step 1: Creat tools

In [ ]:
@tool
def add(a: int, b: int) -> int:
    """Add two numbers"""
    print(f"Adding {a} + {b}")
    return a + b


@tool
def sub(a: int, b: int) -> int:
    """Substract two numbers"""
    print(f"Substracting {a} - {b}")
    return a - b

### Step 2: Create a list and a map of tools

In [ ]:
tools = [add, sub]
tools_map = {t.name.lower(): t for t in tools}
tools_map

### Step 3: Bind tools to the LLM, invoke it

In [ ]:
# Create the LLM object
llm = ChatOllama(model=MODEL, base_url=OLLAMA_URL)

# Bind the tools to the LLM
llm_with_tools = llm.bind_tools(tools)

# Create a message
query = "What is the addition of 40 and 2?"
message = HumanMessage(query)
messages = [message]                # The list of messages to send to the LLM, now is only one message

ret = llm_with_tools.invoke(messages)
print(f"Type: {type(ret)}\n{ret}")

In [ ]:
# Take a look at the tool calls
ret.tool_calls

### Step 4: Call the functions the LLM suggests

In [ ]:
# Call the tools, append the results to the messages list
for tool_call in ret.tool_calls:
    tool_name = tool_call['name'].lower()
    tool = tools_map.get(tool_name)
    if tool:
        tool_msg = tool.invoke(tool_call)
        print(tool_msg)
    print(f"Tool message: {tool_msg}")
    messages.append(tool_msg)

# Show the messages
messages

### Step 5: Append the tool's output, invoke the LLM again

In [ ]:
# Invoke the LLM with the messages (including the tool results)
ret = llm_with_tools.invoke(messages)
print(ret.content)